# CP 322 - Final Project

### Names & ID

- Aditya Chauhan (169027493)
- Matthew McCulley ()

### Dataset Choice

- [Student Performance Dataset](https://archive.ics.uci.edu/dataset/320/student+performance)

### Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Read Data

In [3]:
data = pd.read_csv("student-mat.csv", sep=";")

## Data Quality Report

#### Dataset Shape:

The shape of the data's distribution can help with finding descriptive statistics.

#### Dataset Columns:

The columns of a dataset (Descriptive & Target Features).

#### Missing Values

In [ ]:
print("====== Data Quality report ======")

def generate_data_quality_report(df):
    print("\n Dataset Shape: ", df.shape)
    print("\n Columns: ", df.columns.tolist())
    
        